In [1]:
import pandas as pd

# Helpers
from data.data_builder import alfred_dataset
from models.models_builder import asset_arima, nowcaster

import statsmodels.formula.api as smf

from statsmodels.tools.eval_measures import rmse

%reload_ext autoreload

# ***1 - Data***

In [2]:
indices = ["CPI", "Core CPI", "Food CPI", "Gasoline CPI"]
df = alfred_dataset(indices)
df.describe().T

,count,mean,min,25%,50%,75%,max,std
date,688,1995-08-16 16:53:01.395348864,1967-01-01 00:00:00,1981-04-23 12:00:00,1995-08-16 12:00:00,2009-12-08 18:00:00,2024-04-01 00:00:00,NaN
CPI,688.0,151.966017,32.9,89.55,153.0,217.3655,313.207,76.740787
Core CPI,688.0,156.092715,34.2,87.6,162.0,220.68225,317.622,78.315558
Food CPI,688.0,153.538073,33.7,92.8,148.85,219.01,328.267,78.17727
Gasoline CPI,688.0,139.073121,26.1,80.4,102.1,211.595,411.984,90.836301


In [3]:
pd.read_excel("data/2e. RBRTEd.xls", sheet_name=1, header=2)

,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
...,...,...
9374,2024-04-29,88.44
9375,2024-04-30,88.23
9376,2024-05-01,83.55
9377,2024-05-02,84.81


In [4]:
pd.read_excel("data/2f. EMM_EPMR_PTE_NUS_DPGw.xls", sheet_name=1, header=2)

,Date,Weekly U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)
0,1990-08-20,1.191
1,1990-08-27,1.245
2,1990-09-03,1.242
3,1990-09-10,1.252
4,1990-09-17,1.266
...,...,...
1755,2024-04-08,3.591
1756,2024-04-15,3.628
1757,2024-04-22,3.668
1758,2024-04-29,3.653


# ***2 - Model***

### 2.1 - MAs

In [4]:
ccpi_ma, ccpi_ma_res = asset_arima(df, 'Core CPI')
df['Core CPI MA'] = ccpi_ma_res.fittedvalues

/Users/paul/opt/miniconda3/envs/ds/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:               Core CPI   No. Observations:                  688
Model:                ARIMA(0, 0, 12)   Log Likelihood               -3211.511
Date:                Wed, 15 May 2024   AIC                           6451.021
Time:                        16:53:31   BIC                           6514.494
Sample:                             0   HQIC                          6475.577
                                - 688                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        156.0915      5.420     28.799      0.000     145.469     166.714
ma.L1          5.4129      0.581      9.316      0.000       4.274       6.552
ma.L2         14.3085      3.759      3.807      0.0

In [5]:
fcpi_ma, fcpi_ma_res = asset_arima(df, 'Food CPI')
df['Food CPI MA'] = fcpi_ma_res.fittedvalues

/Users/paul/opt/miniconda3/envs/ds/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:               Food CPI   No. Observations:                  688
Model:                ARIMA(0, 0, 12)   Log Likelihood               -2955.149
Date:                Wed, 15 May 2024   AIC                           5938.297
Time:                        16:53:43   BIC                           6001.770
Sample:                             0   HQIC                          5962.853
                                - 688                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        153.0514      2.014     75.992      0.000     149.104     156.999
ma.L1          5.3842      1.576      3.416      0.001       2.295       8.473
ma.L2         14.6175     10.069      1.452      0.1

/Users/paul/opt/miniconda3/envs/ds/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [6]:
gcpi_ma, gcpi_ma_res = asset_arima(df, 'Gasoline CPI')
df['Gasoline CPI MA'] = gcpi_ma_res.fittedvalues

/Users/paul/opt/miniconda3/envs/ds/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:           Gasoline CPI   No. Observations:                  688
Model:                ARIMA(0, 0, 12)   Log Likelihood               -2667.118
Date:                Wed, 15 May 2024   AIC                           5362.237
Time:                        16:53:57   BIC                           5425.710
Sample:                             0   HQIC                          5386.792
                                - 688                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        139.0775      9.106     15.274      0.000     121.231     156.924
ma.L1          1.4341      0.033     44.119      0.000       1.370       1.498
ma.L2          1.6430      0.045     36.542      0.0

### 2.2 - Final OLS

In [8]:
df

,date,CPI,Core CPI,Food CPI,Gasoline CPI,Core CPI MA,Food CPI MA,Gasoline CPI MA
0,1967-01-01,32.900,34.200,33.900,26.300,156.091462,153.051373,139.077510
1,1967-02-01,33.000,34.200,33.800,26.200,54.054665,54.590886,30.498156
2,1967-03-01,33.000,34.300,33.800,26.100,72.042608,72.995694,32.531475
3,1967-04-01,33.100,34.400,33.700,26.400,42.796457,43.384516,33.870286
4,1967-05-01,33.100,34.500,33.700,26.400,49.813653,50.798055,35.190520
...,...,...,...,...,...,...,...,...
683,2023-12-01,308.742,313.209,326.545,303.242,303.970133,279.167287,295.072201
684,2024-01-01,309.685,314.438,327.813,293.287,246.063220,296.916543,297.225863
685,2024-02-01,311.054,315.565,327.880,304.302,319.153929,299.265503,262.894374
686,2024-03-01,312.230,316.698,328.212,309.587,246.973962,287.793693,310.040198


In [12]:
X = ["Core CPI MA", "Food CPI MA", "Gasoline CPI MA"]
y = "CPI"
start_date = "2000-01-01"

In [13]:
nowcaster_fed, nowcaster_fed_res = nowcaster(df, X, y, start=start_date)

                            OLS Regression Results                            
Dep. Variable:                    CPI   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     1148.
Date:                Wed, 15 May 2024   Prob (F-statistic):          8.44e-160
Time:                        16:55:25   Log-Likelihood:                -1085.9
No. Observations:                 292   AIC:                             2180.
Df Residuals:                     288   BIC:                             2195.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -47.2522      5.281     

In [14]:
yh = nowcaster_fed_res.fittedvalues

In [14]:
rmse(y, yh)

10.909518003632684

In [20]:
res_df = pd.DataFrame({'Realized CPI': y, 'Forecast CPI': yh})
res_df

,Realized CPI,Forecast CPI
0,32.900,145.473350
1,33.000,35.323420
2,33.000,53.036286
3,33.100,25.091264
4,33.100,32.227820
...,...,...
683,308.742,294.064851
684,309.685,288.668253
685,311.054,307.266438
686,312.230,284.759007


In [15]:
yh.pct_change()

0           NaN
1     -0.757183
2      0.501448
3     -0.526904
4      0.284424
         ...   
683   -0.038451
684   -0.018352
685    0.064428
686   -0.073251
687    0.093074
Length: 688, dtype: float64

In [14]:
spf = pd.read_excel('data/SPF_Median_CPI.xlsx')
spf

/Users/paul/opt/miniconda3/envs/ds/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,YEAR,QUARTER,CPI1,CPI2,CPI3,CPI4,CPI5,CPI6,CPIA,CPIB,CPIC
0,1968,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1969,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1969,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1969,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
218,2023,2,3.8,3.5000,3.2000,2.9492,2.7000,2.4168,3.3519,2.5118,2.2500
219,2023,3,2.7,3.0680,2.9230,2.6146,2.5493,2.5920,3.1000,2.4862,2.3981
220,2023,4,3.6,3.2641,2.7832,2.6159,2.5000,2.4000,3.3373,2.5193,2.2707
221,2024,1,2.8,2.5094,2.5000,2.4000,2.3547,2.2677,2.4905,2.2000,2.2523
